In [1]:
!pip install opencv-python
!pip install scikit-learn
!pip install scikit-image
!pip install seaborn

     |████████████████████████████████| 32.6 MB 10.5 MB/s eta 0:00:01    |██████████▍                     | 10.6 MB 6.7 MB/s eta 0:00:04


In [2]:
import cv2
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import skimage
from sklearn.cluster import KMeans
import math
import seaborn as sns


def img_prepro(img):
    dim = (112, 112)
    resized_img = cv2.resize(img, dim)
    gaussian_blur_img = cv2.GaussianBlur(resized_img, (5,5),10)
    gaussian_blur_img2= cv2.GaussianBlur(gaussian_blur_img, (5,5),10)
    crop_img = gaussian_blur_img2[:23, :26]

    sv_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2HSV)
    gaussian_blur_img = cv2.GaussianBlur(sv_img, (5,5),10)

    return gaussian_blur_img


def leer_paths(path):
    files = [ os.path.join(path, file) for file in os.listdir(path) if file.endswith(".jpg") ]
    return files


def convertir_a_imagenes(imgs):
    return [cv2.imread(x) for x in imgs]


X_files = leer_paths("../zip/train/")
Y_files = leer_paths("../zip/test/")
Y_files = sorted(Y_files, key=lambda name: int(name.split("/")[-1][:-4]))

train_data = convertir_a_imagenes(X_files)
test_data = convertir_a_imagenes(Y_files)

train_data = [ img_prepro(img) for img in train_data ]
test_data = [ img_prepro(img) for img in test_data ]

# NORMALIZAR LOS DATOS (pasar de 0-255 a 0-1)

#for index in range(len(test_data)):
    #test_data[index] = test_data[index] / 255.0
    
# for index in range(len(train_data)):
    #train_data[index] = train_data[index] / 255.0

FileNotFoundError: FileNotFoundError: [Errno 2] No such file or directory: '../zip/train/'

In [ ]:
#MOSTRAR 

plt.figure()
plt.imshow(train_data[0])
plt.colorbar()
plt.grid(False)
plt.show()
#contours = cv2.findContours(resized_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#cv2.drawContours(resized_img, contours[0], -1, (0,255.0), 3)
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_data[i], cmap=plt.cm.binary)
plt.show()

<Figure size 640x480 with 2 Axes>

<Figure size 1000x1000 with 25 Axes>

In [ ]:
from sklearn.decomposition import PCA
train_data_array = np.array(train_data)
test_data_array = np.array(test_data)

train_array2d_2 = []
test_array2d_2 = []
for i in range(len(train_data_array)):
    train_array2d_2.append(train_data_array[i].flatten())
for i in range(len(test_data_array)):
    test_array2d_2.append(test_data_array[i].flatten())
    
#pca = PCA(n_components = 1)
#pca.fit(train_array2d_2)
#train_array2d = pca.transform(train_array2d_2)
#test_array2d = pca.transform(test_array2d_2)
train_array2d = train_array2d_2
test_array2d = test_array2d_2

In [ ]:
kmeans = KMeans(n_clusters = 5, random_state = 42)
kmeans.fit(train_array2d)
labelsKmeans = kmeans.predict(test_array2d)

C:\Users\marco\AppData\Roaming\jupyterlab-desktop\jlab_server\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
from sklearn.cluster import AgglomerativeClustering

agglomerative_cluster = AgglomerativeClustering(n_clusters=5)
labelsAgglomerative = agglomerative_cluster.fit_predict(np.concatenate((train_array2d, test_array2d)))

labelsAgglomerative = labelsAgglomerative[-len(test_array2d):]

In [ ]:
from sklearn.cluster import SpectralClustering

spectral_cluster = SpectralClustering(assign_labels='discretize', n_clusters=5, random_state=42, n_jobs = -1)
labelsSpectral = spectral_cluster.fit_predict(np.concatenate((train_array2d, test_array2d)))

labelsSpectral = labelsSpectral[-len(test_array2d):]

C:\Users\marco\AppData\Roaming\jupyterlab-desktop\jlab_server\lib\site-packages\sklearn\manifold\_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


In [ ]:
from sklearn.metrics.cluster import adjusted_rand_score

for i in np.arange(0.1, 2.0, 0.1):
    spectral_cluster = SpectralClustering(assign_labels=['discretize', 'kmeans'], n_clusters=5, random_state=42, n_init = 100, gamma = i)
    labels = spectral_cluster.fit_predict(test_array2d)


In [ ]:
num_id = []
for filename in os.listdir("../zip/test/"):
    punto = filename.index('.')
    cadena = filename[0:punto]
    num_id.append(int(cadena))

ids = pd.DataFrame(num_id)
clusters = pd.DataFrame(labelsKmeans)
clusters = clusters.astype(str)


solucion = ids.join(clusters, how='left', lsuffix='id', rsuffix='suit')
solucion.columns = ['id', 'suit']
print("csv done")
solucion.to_csv('ficheroKmeans.csv', index=False) 

csv done


In [ ]:
from sklearn.metrics.cluster import adjusted_rand_score

modelos = ["kmeans", "agglomerate", "spectral"]
ari_results = [ariKmeans, ariAgglomerative, ariSpectral]
df = pd.DataFrame()
df.insert(0, "modelos", modelos)
df.insert(1, "Ari", ari_results)
display(df)

sns.barplot(x="Ari", y="modelos", data=df, palette="pastel")

,modelos,Ari
0,kmeans,0.171241
1,agglomerate,0.160327
2,spectral,-0.000964


       modelos       Ari
0       kmeans  0.171241
1  agglomerate  0.160327
2     spectral -0.000964

<AxesSubplot: xlabel='Ari', ylabel='modelos'>

<Figure size 640x480 with 1 Axes>